## Questions to Answer
        
        ## 1.Which brewery produces the strongest beers by ABV%?
## 2.If you had to pick 3 beers to recommend using only this data, which would you pick?
        ## 3. Which of the factors (aroma, taste, appearance, palette) are most important in determining the overall quality of a beer?
        ## 4.Lastly, if I typically enjoy a beer due to its aroma and appearance, which beer style should I try?
        
        
### All of the Null Treatments and other subsettings have been copied from the EDA.ipynb jupyter notebook Please refer to that if you find any confusions


In [1]:
## Loading the important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import sklearn
%matplotlib inline


### **Here i am trying to build a very basic similarity based recommeder system**

In [2]:
df_raw = pd.read_csv('C:/Lean_Skill_InterviewTest/data/beer-review-data-set.csv')
df_raw["user_birthdayRaw"].fillna(0,inplace= True)
df_raw["review_text"].fillna(0,inplace= True)
df_raw["user_birthdayRaw"].fillna('unknwn',inplace= True)
df_raw["user_birthdayUnix"].fillna(0,inplace= True)
df_raw["user_gender"].fillna('unknwn',inplace= True)
df_raw["user_ageInSeconds"].fillna('0',inplace= True)
df_raw["user_profileName"].fillna('unknwn',inplace= True)
df_raw.isnull().sum()
#df_raw.columns
### Reading the data and Treating all the null values in the data frame 
df_raw.describe()

,index,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_aroma,review_overall,review_palate,review_taste,review_timeUnix,user_birthdayUnix
count,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.00000,37500.000000,37500.000000,3.750000e+04,3.750000e+04
mean,24951.887573,7.403725,21861.152027,3036.595120,3.900053,3.873240,3.88944,3.854867,3.922440,1.232794e+09,5.061995e+07
std,14434.009669,2.318145,18923.130832,5123.084675,0.588778,0.680865,0.70045,0.668068,0.716504,7.190955e+07,1.831302e+08
min,0.000000,0.100000,175.000000,1.000000,0.000000,1.000000,0.00000,1.000000,1.000000,9.262944e+08,-2.208960e+09
25%,12422.500000,5.400000,5441.000000,395.000000,3.500000,3.500000,3.50000,3.500000,3.500000,1.189194e+09,0.000000e+00
50%,24942.500000,6.900000,17538.000000,1199.000000,4.000000,4.000000,4.00000,4.000000,4.000000,1.248150e+09,0.000000e+00
75%,37416.750000,9.400000,34146.000000,1315.000000,4.500000,4.500000,4.50000,4.500000,4.500000,1.291330e+09,0.000000e+00
max,49999.000000,57.700000,77207.000000,27797.000000,5.000000,5.000000,5.00000,5.000000,5.000000,1.326267e+09,7.148988e+08


In [3]:
Highly_reviewed=pd.DataFrame(df_raw['user_profileName'].value_counts().reset_index().head(30))
Highly_reviewed[Highly_reviewed['user_profileName']>1]

,index,user_profileName
0,northyorksammy,141
1,BuckeyeNation,110
2,mikesgroove,92
3,ChainGangGuy,84
4,MasterSki,83
5,akorsak,81
6,oberon,81
7,barleywinefiend,80
8,Thorpe429,79
9,smcolw,74


#### Based on this we can say that there are only 4285 people who have given more than 1 review all the other reviews are single reviews 

In [4]:
#***review_appearance 	review_aroma 	review_overall 	review_palate 	review_taste***
#*** 3.900053 	3.873240 	3.88944 	3.854867 	3.922440 	***

In [5]:
df_raw.columns

Index(['index', 'beer_ABV', 'beer_beerId', 'beer_brewerId', 'beer_name',
       'beer_style', 'review_appearance', 'review_aroma', 'review_overall',
       'review_palate', 'review_taste', 'review_text', 'review_timeStruct',
       'review_timeUnix', 'user_ageInSeconds', 'user_birthdayRaw',
       'user_birthdayUnix', 'user_gender', 'user_profileName'],
      dtype='object')



#### **grouping all the beers based on their ratings**



In [6]:
review_appearance_ratings = pd.DataFrame(df_raw.groupby('beer_name')['review_appearance'].mean()).reset_index().rename(columns={'beer_name':'beer_beerId','review_appearance' : 'review_appearance_avg'})
review_appearance_ratings.head(10).sort_values('review_appearance_avg',ascending = False)

,beer_beerId,review_appearance_avg
4,10 Year Clelebration Ale,4.000000
5,12 Year Anniversary Ale,4.000000
7,16 Year Anniversary Ale,4.000000
8,1871 Lager,4.000000
0,"""The Wind Cried Mari..."" Scottish Heather Ale",3.875000
2,1 A.M. Ale,3.833333
3,10 Blocks South,3.500000
6,13th Anniversary Imperial India Pale Ale,3.500000
9,1906 Reserva Especial,3.307692
1,'Pooya Porter,3.000000


In [7]:
review_aroma_ratings = pd.DataFrame(df_raw.groupby('beer_name')['review_aroma'].mean()).reset_index().rename(columns={'beer_name':'beer_name','review_aroma' : 'review_aroma_avg'})
review_aroma_ratings.head(10).sort_values('review_aroma_avg',ascending = False)

,beer_name,review_aroma_avg
5,12 Year Anniversary Ale,4.500000
7,16 Year Anniversary Ale,4.500000
0,"""The Wind Cried Mari..."" Scottish Heather Ale",4.175000
1,'Pooya Porter,4.000000
8,1871 Lager,3.750000
2,1 A.M. Ale,3.666667
3,10 Blocks South,3.500000
4,10 Year Clelebration Ale,3.500000
6,13th Anniversary Imperial India Pale Ale,3.500000
9,1906 Reserva Especial,2.730769


In [8]:
review_palate_ratings = pd.DataFrame(df_raw.groupby('beer_name')['review_palate'].mean()).reset_index().rename(columns={'beer_name':'beer_beerId','review_palate' : 'review_palate_avg'})
review_palate_ratings.head(10).sort_values('review_palate_avg',ascending = False)

,beer_beerId,review_palate_avg
7,16 Year Anniversary Ale,4.250000
4,10 Year Clelebration Ale,4.000000
5,12 Year Anniversary Ale,4.000000
6,13th Anniversary Imperial India Pale Ale,4.000000
0,"""The Wind Cried Mari..."" Scottish Heather Ale",3.925000
8,1871 Lager,3.750000
1,'Pooya Porter,3.500000
3,10 Blocks South,3.500000
9,1906 Reserva Especial,3.192308
2,1 A.M. Ale,3.000000


In [9]:
review_taste_ratings = pd.DataFrame(df_raw.groupby('beer_name')['review_taste'].mean()).reset_index().rename(columns={'beer_name':'beer_beerId','review_taste' : 'review_taste_avg'})
review_taste_ratings.head(10).sort_values('review_taste_avg',ascending = False)

,beer_beerId,review_taste_avg
4,10 Year Clelebration Ale,4.500000
0,"""The Wind Cried Mari..."" Scottish Heather Ale",4.050000
1,'Pooya Porter,4.000000
3,10 Blocks South,4.000000
5,12 Year Anniversary Ale,4.000000
7,16 Year Anniversary Ale,4.000000
8,1871 Lager,4.000000
6,13th Anniversary Imperial India Pale Ale,3.500000
9,1906 Reserva Especial,2.923077
2,1 A.M. Ale,2.666667


### **Just based out of the ratings data**
1.) 16 Year Anniversary Ale
 2.) 12 Year Anniversary Ale
 3.) 10 Year Clelebration Ale

**these beers come with some of the highest ratings in diffrent ratings**
 
#### **But this is a crude way of doing things**

### **Lets try out the recommender system here**

Lets try by taking the average of all the ratings , ive done that in the EDA portion and described how it is a nice approach as considered to the individual ratings approach under my umbrella of assumptions

In [10]:
df_raw['total_rating'] = df_raw.apply(lambda x: x['review_aroma']+x['review_appearance'] + x['review_taste']+x['review_palate']+x['review_overall'], axis=1)
#df_raw['avg_rating']=df_raw['avg_rating']/5
df_raw['total_rating']

0        20.0
1        17.5
2        18.0
3        14.5
4        16.0
5        20.5
6        20.5
7        20.0
8        22.0
9        10.5
10       22.5
11       20.5
12       23.5
13       11.5
14       19.5
15       18.5
16       17.0
17       19.0
18       20.0
19       18.5
20       21.5
21       21.0
22       21.0
23       20.5
24       20.0
25       19.5
26       21.5
27       21.0
28       19.5
29       19.0
         ... 
37470    18.0
37471    21.5
37472    17.0
37473    20.5
37474    15.0
37475    20.5
37476    19.0
37477    21.0
37478    21.0
37479    19.5
37480    17.5
37481    18.5
37482    21.5
37483    19.0
37484    19.0
37485    18.5
37486    20.0
37487    19.5
37488    25.0
37489    21.0
37490    17.0
37491    22.0
37492    16.0
37493    11.5
37494    20.5
37495    18.0
37496    21.0
37497    20.0
37498    22.0
37499    20.5
Name: total_rating, Length: 37500, dtype: float64

In [11]:
df_raw.describe()

,index,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_aroma,review_overall,review_palate,review_taste,review_timeUnix,user_birthdayUnix,total_rating
count,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.00000,37500.000000,37500.000000,3.750000e+04,3.750000e+04,37500.000000
mean,24951.887573,7.403725,21861.152027,3036.595120,3.900053,3.873240,3.88944,3.854867,3.922440,1.232794e+09,5.061995e+07,19.440040
std,14434.009669,2.318145,18923.130832,5123.084675,0.588778,0.680865,0.70045,0.668068,0.716504,7.190955e+07,1.831302e+08,2.822563
min,0.000000,0.100000,175.000000,1.000000,0.000000,1.000000,0.00000,1.000000,1.000000,9.262944e+08,-2.208960e+09,5.000000
25%,12422.500000,5.400000,5441.000000,395.000000,3.500000,3.500000,3.50000,3.500000,3.500000,1.189194e+09,0.000000e+00,18.000000
50%,24942.500000,6.900000,17538.000000,1199.000000,4.000000,4.000000,4.00000,4.000000,4.000000,1.248150e+09,0.000000e+00,20.000000
75%,37416.750000,9.400000,34146.000000,1315.000000,4.500000,4.500000,4.50000,4.500000,4.500000,1.291330e+09,0.000000e+00,21.500000
max,49999.000000,57.700000,77207.000000,27797.000000,5.000000,5.000000,5.00000,5.000000,5.000000,1.326267e+09,7.148988e+08,25.000000


In [12]:
columns = ['beer_beerId', 'user_profileName', 'total_rating']
beer_rec = df_raw[columns]
beer_rec.head()

,beer_beerId,user_profileName,total_rating
0,46634,RblWthACoz,20.0
1,3003,BeerSox,17.5
2,961,mschofield,18.0
3,429,molegar76,14.5
4,4904,Brewbro000,16.0


In [13]:
ratings_pivot = beer_rec.pivot_table(index=['user_profileName'], columns=['beer_beerId'], values='total_rating')
ratings_pivot.head()

beer_beerId,175,176,178,429,436,454,503,505,507,508,...,76963,76995,76996,76997,76998,76999,77116,77198,77199,77207
user_profileName,,,,,,,,,,,,,,,,,,,,,
0110x011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05Harley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0tt0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100floods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1099,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
### Lets just take the most commonly consumed beer as the benchmark for our rating syatem 
#Founders Breakfast Stout

In [15]:
df_raw[df_raw['beer_name'].str.contains('Aecht Schlenkerla Faste.')].shape

(62, 20)

In [16]:
df_raw[df_raw['beer_name'].str.contains('Aecht Schlenkerla Faste.')]['beer_name'].iloc[0]

'Aecht Schlenkerla Fastenbier'

In [17]:
df_raw[df_raw['beer_name'].str.contains('Aecht Schlenkerla Faste.')]['beer_beerId'].iloc[0]

21704

In [18]:
gf_30th_ratings = ratings_pivot[21704]
gf_30th_ratings

user_profileName
0110x011           NaN
05Harley           NaN
0tt0               NaN
100floods          NaN
1099               NaN
11osixBrew         NaN
12puebloyankee     NaN
12vUnion           NaN
1759Girl           NaN
184601             NaN
1993Heel           NaN
1Adam12            NaN
1fastz28           NaN
1morebeer          NaN
1p3261             NaN
1santore           NaN
1thinmint          NaN
1whiskey           NaN
2378GCGTG          NaN
2ndstage           NaN
2roosters          NaN
2xHops             NaN
31thadrty1         NaN
321jeff            NaN
33nickadams        NaN
3HovsepiansBrews   NaN
3Vandoo            NaN
3dudesbeerblog     NaN
4000qtrap          NaN
404UGA             NaN
                    ..
zappuh             NaN
zbellino           NaN
zbuc919            NaN
zchall23           NaN
zdk9               NaN
zebedee            NaN
zeff80             NaN
zeldar             NaN
zeledonia          NaN
zeomally           NaN
zep9865            NaN
zerk             

##### Based on the above data i can say that there are a lot of review of this beer lets take any beer between 4.3 and 4.5 since that subset of beers is large and will help us in finding better recommendations. 

## Aecht Schlenkerla Fastenbier	4.3	62


In [19]:
#### Lets try to find the correlation between the beers and see what scores come up

In [20]:
similar_beers = ratings_pivot.corrwith(gf_30th_ratings)
similar_beers.dropna()
similar_beers_df = pd.DataFrame(similar_beers)
similar_beers_df.head(20)

c:\python36\lib\site-packages\numpy\lib\function_base.py:2392: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
c:\python36\lib\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,0
beer_beerId,
175,-0.060206
176,0.840328
178,0.596040
429,0.313459
436,0.424238
454,-0.934720
503,NaN
505,NaN
507,-1.000000


In [21]:
similar_beers.dropna(inplace=True)
similar_beers_df.dropna(inplace=True)
similar_beers_df.shape

(179, 1)

In [22]:
# Now to sort the values, based on similarity score
similar_beers_df.sort_values(by=0,ascending=False)


,0
beer_beerId,
13350,1.000000
17655,1.000000
35717,1.000000
1589,1.000000
46634,1.000000
1791,1.000000
7047,1.000000
562,1.000000
5756,1.000000


### High correlation between beers 

In [23]:
beer_stats = beer_rec.groupby('beer_beerId').agg({'total_rating': [np.size, np.mean]})
beer_stats.head()

total_rating           
                    size       mean
beer_beerId                        
175                 56.0  17.142857
176                 92.0  18.961957
178                 68.0  17.419118
429                947.0  18.294615
436                345.0  12.484058

In [24]:
beer_stats.describe()

total_rating             
              size         mean
count  1731.000000  1731.000000
mean     21.663778    18.042703
std      99.528578     2.669383
min       1.000000     5.000000
25%       1.000000    16.650000
50%       2.000000    18.500000
75%       7.000000    20.000000
max    1883.000000    24.000000

### Average rating a beer has is 22 , very high standard deviation , this again is not helping our problem statement lets make a subset of all the beers with a rating of more than 500

In [25]:
popular_beers = beer_stats['total_rating']['size'] >= 75
beer_stats[popular_beers].sort_values([('total_rating', 'mean')], ascending=False)[:15]

total_rating           
                    size       mean
beer_beerId                        
47658              482.0  22.840249
19960             1441.0  22.441013
11757             1883.0  22.025757
54827              115.0  21.486957
21822              805.0  21.410559
33644              636.0  21.083333
47695              298.0  20.998322
57110              182.0  20.980769
7348               675.0  20.957037
16074             1051.0  20.931018
51483               98.0  20.709184
34146              857.0  20.663361
20478              492.0  20.662602
25852              275.0  20.603636
1504               381.0  20.577428

In [26]:
beer_rec_trimmed = beer_stats[popular_beers].join(pd.DataFrame(similar_beers, columns=['similarity']))
beer_rec_trimmed

c:\python36\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(total_rating, size)","(total_rating, mean)",similarity
beer_beerId,,,
176,92.0,18.961957,0.840328
429,947.0,18.294615,0.313459
436,345.0,12.484058,0.424238
727,665.0,20.065414,0.388696
960,154.0,18.860390,0.483429
961,78.0,17.532051,-0.631266
962,146.0,17.828767,-0.602843
966,206.0,20.262136,0.481969
1018,252.0,19.341270,0.525846


In [27]:
print(beer_stats[popular_beers].shape)
print(similar_beers.shape)
print(beer_rec_trimmed.shape)
similar_beers.isnull().sum()

(86, 2)
(179,)
(86, 3)


0

In [28]:
beer_rec_trimmed.sort_values(['similarity'], ascending=False)[:15]

,"(total_rating, size)","(total_rating, mean)",similarity
beer_beerId,,,
1791,83.0,18.566265,1.000000
13350,155.0,17.987097,1.000000
16638,133.0,15.924812,1.000000
2106,179.0,19.606145,0.933528
2572,77.0,19.441558,0.933035
176,92.0,18.961957,0.840328
8422,91.0,20.076923,0.792501
1122,119.0,19.046218,0.670554
52211,139.0,18.442446,0.599072


In [29]:
# Convert the index into a list for use when searching raw dataframe.
rec_beer_ids = beer_rec_trimmed.sort_values(['similarity'], ascending=False)[:25].index.tolist()
rec_beer_ids

[1791,
 13350,
 16638,
 2106,
 2572,
 176,
 8422,
 1122,
 52211,
 57110,
 37518,
 43312,
 1018,
 23474,
 7324,
 960,
 966,
 5411,
 436,
 47695,
 51483,
 727,
 16073,
 54731,
 4904]

### Here we have all the top maching beer ID's with their respective 

In [30]:
rec_beer_names = df_raw[df_raw.beer_beerId.isin(rec_beer_ids)]['beer_name'].unique().tolist()
rec_breweries = df_raw[df_raw.beer_beerId.isin(rec_beer_ids)]['beer_brewerId'].unique().tolist()
rec_beer_df = pd.DataFrame({'beer_beerId':rec_beer_ids, 'beer_name':rec_beer_names})

In [31]:
### Finiding the similar beer IDs with high similarity 
beers_ids= beer_rec_trimmed.sort_values(['similarity'], ascending=False)[:25].reset_index()
beers_ids.head(25)

,beer_beerId,"(total_rating, size)","(total_rating, mean)",similarity
0,1791,83.0,18.566265,1.000000
1,13350,155.0,17.987097,1.000000
2,16638,133.0,15.924812,1.000000
3,2106,179.0,19.606145,0.933528
4,2572,77.0,19.441558,0.933035
5,176,92.0,18.961957,0.840328
6,8422,91.0,20.076923,0.792501
7,1122,119.0,19.046218,0.670554
8,52211,139.0,18.442446,0.599072
9,57110,182.0,20.980769,0.584732


In [32]:
rec_beer_df = pd.merge(rec_beer_df, beers_ids, on='beer_beerId')
rec_beer_df

,beer_beerId,beer_name,"(total_rating, size)","(total_rating, mean)",similarity
0,1791,Black Sheep Ale (Special),83.0,18.566265,1.000000
1,13350,Frog's Hollow Double Pumpkin Ale,155.0,17.987097,1.000000
2,16638,Pike India Pale Ale,133.0,15.924812,1.000000
3,2106,Pumpkin Ale,179.0,19.606145,0.933528
4,2572,Founders RÃ¼bÃ¦us,77.0,19.441558,0.933035
5,176,Aecht Schlenkerla Rauchbier MÃ¤rzen,92.0,18.961957,0.840328
6,8422,Sue,91.0,20.076923,0.792501
7,1122,Batemans XXXB,119.0,19.046218,0.670554
8,52211,Pride Of Milford Special Ale,139.0,18.442446,0.599072
9,57110,Aecht Schlenkerla Rauchbier Weizen,182.0,20.980769,0.584732


## Top Three beer recomendations 
#1. Black Sheep Ale (Special) ; brewery_ID:1417
#2. Founders RÃ¼bÃ¦us ; brewery_ID:1199
#3.Aecht Schlenkerla Rauchbier MÃ¤rzen ; brewery_ID:263

## They have almost similar avegare ratings and almost similar Total ratings which near in the limit of 75 to 95 and also based on similarity scores